# Laboratorio 1 (AlpesHearth): Exploración, preparación y regresión lineal

Este notebook está diseñado para **resolver el enunciado completo** del Laboratorio 1, siguiendo el estilo y las ideas de:
- `ManipulacionExploracionCalidad.ipynb` (exploración + calidad: completitud, unicidad, consistencia, validez)
- `LimpiezaTransformacion.ipynb` (limpieza + transformación)
- `RegresionLineal.ipynb` (pipeline, métricas, y verificación de supuestos)

**Objetivo técnico del laboratorio:** predecir `CVD Risk Score` (variable continua) con modelos de regresión lineal, comparar al menos 2 estrategias de preparación, interpretar coeficientes del mejor modelo y generar predicciones para el set de test no etiquetado.

Nota de compatibilidad: `Datos Lab 1.csv` usa separador `,` y `Datos Test Lab 1.csv` usa separador `;`. Este notebook lo maneja automáticamente.


## 0. Imports y constantes

Qué haces aquí: importar librerías y fijar constantes exigidas por el enunciado.

Por qué existe esta celda: evita imports repetidos y hace explícitos los parámetros obligatorios: `random_state=42` y `test_size=0.25`.


In [1]:
import pandas as pd
import numpy as np
import re

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

from sklearn.base import BaseEstimator, TransformerMixin

pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 140)

RANDOM_STATE = 42
TEST_SIZE = 0.25

TARGET = "CVD Risk Score"
AUX_LABEL = "CVD Risk Level"


## 1. Cargar datos (train, test) y diccionario

Qué haces aquí:
1. Leer los CSV de train y test detectando separador automáticamente.
2. Leer el diccionario en Excel para entender el significado de las columnas.

Por qué existe esta celda:
- El enunciado exige revisar el diccionario primero.
- Si lees el test con separador incorrecto, todo el notebook falla.


In [ ]:

def read_csv_auto_sep(path: str) -> pd.DataFrame:
    # Detecta si el separador es ; o , leyendo la primera línea del archivo
    with open(path, "r", encoding="utf-8", errors="ignore") as f:
        first_line = f.readline()
    sep = ";" if first_line.count(";") > first_line.count(",") else ","
    return pd.read_csv(path, sep=sep)

train_path = "Datos Lab 1.csv"
test_path  = "Datos Test Lab 1.csv"
dicc_path  = "DiccPacientes.xlsx"

train_raw = read_csv_auto_sep(train_path)
test_raw  = read_csv_auto_sep(test_path)
dicc_raw  = pd.read_excel(dicc_path)

# El diccionario trae el nombre de columna con un espacio final, lo normalizamos.
dicc = dicc_raw.copy()
dicc.columns = [c.strip() for c in dicc.columns]

print("Train shape:", train_raw.shape)
print("Test  shape:", test_raw.shape)
display(dicc.head(10))

# Validación rápida: train debe tener target, test no.
print("Columnas extra en train (vs test):", sorted(list(set(train_raw.columns) - set(test_raw.columns))))


## 2. Exploración inicial (descripción básica)

Qué haces aquí:
- Mirar primeras filas (`head`), tipos (`dtypes`) y resumen estadístico (`describe`).

Por qué existe esta celda:
- Es el punto de partida para detectar problemas de calidad y decidir preparación.
- Esto sigue el enfoque de "Descripción de los datos" en los notebooks guía.


In [ ]:

display(train_raw.head())
display(test_raw.head())

print("Tipos de datos (train):")
display(train_raw.dtypes)

print("Describe numérico (train):")
display(train_raw.describe(numeric_only=True))

print("Describe del target (train):")
display(train_raw[TARGET].describe())


## 3. Calidad de datos (completitud, unicidad, consistencia, validez)

Esta sección replica el esquema de los notebooks:
- Completitud: porcentaje de faltantes por columna.
- Unicidad: duplicados exactos.
- Consistencia: categorías con formatos incoherentes (mayúsculas, espacios, etc).
- Validez: valores imposibles o fuera de rango lógico (por ejemplo LDL negativo).

La salida de esta sección es la base para justificar tus decisiones de limpieza.


### 3.1 Completitud (missing values)

Qué haces: calculas el porcentaje de faltantes por columna.

Por qué: si tienes faltantes, debes decidir si imputas, corriges usando otras columnas (por ejemplo BP), o eliminas filas (por ejemplo si falta el target).


In [ ]:

missing_train = (train_raw.isna().mean() * 100).sort_values(ascending=False)
missing_test  = (test_raw.isna().mean() * 100).sort_values(ascending=False)

print("Missing % (train) - top 15")
display(missing_train.head(15))

print("Missing % (test) - top 15")
display(missing_test.head(15))

print("Filas en train con target missing:", train_raw[TARGET].isna().sum())


### 3.2 Unicidad (duplicados)

Qué haces: cuentas duplicados exactos (filas idénticas).

Por qué: duplicados exactos pueden sesgar el entrenamiento porque repites el mismo ejemplo varias veces.


In [ ]:

dup_train = train_raw.duplicated().sum()
dup_test  = test_raw.duplicated().sum()

print("Duplicados exactos train:", dup_train)
print("Duplicados exactos test :", dup_test)


### 3.3 Consistencia (categóricas)

Qué haces: revisas `value_counts()` de columnas categóricas.

Por qué: detecta espacios al inicio/final, mezcla de mayúsculas/minúsculas o etiquetas duplicadas con distinto formato. La regla general es normalizar (strip y formato consistente).


In [ ]:

cat_cols = [
    "Sex",
    "Smoking Status",
    "Diabetes Status",
    "Physical Activity Level",
    "Family History of CVD",
    "Blood Pressure Category",
]

for c in cat_cols:
    print("\n---", c, "---")
    display(train_raw[c].astype(str).value_counts().head(20))


### 3.4 Validez (rangos y valores imposibles)

Qué haces: detectas valores imposibles o fuera de rango lógico.

Por qué: un valor inválido no se debe imputar tal cual. Primero se corrige a `NaN` o se transforma.
Ejemplos claros en estos datos: LDL negativo y colesterol total negativo.


In [ ]:

invalid_ldl = (train_raw["Estimated LDL (mg/dL)"] < 0).sum()
invalid_chol = (train_raw["Total Cholesterol (mg/dL)"] < 0).sum()

print("LDL negativo (train):", invalid_ldl)
print("Colesterol total negativo (train):", invalid_chol)

print("Peso <= 0 (train):", (train_raw["Weight (kg)"] <= 0).sum(skipna=True))
print("Altura (m) <= 0 (train):", (train_raw["Height (m)"] <= 0).sum(skipna=True))
print("Edad <= 0 (train):", (train_raw["Age"] <= 0).sum(skipna=True))


## 4. Limpieza y preparación (decisiones justificadas)

Aquí aplicas decisiones basadas en la sección anterior:
1. Eliminar duplicados exactos en train.
2. Eliminar filas sin target.
3. Validez: convertir LDL negativo y colesterol total negativo a `NaN`.
4. Usar `Blood Pressure (mmHg)` para completar `Systolic BP` y `Diastolic BP` cuando falten.
5. Normalizar categóricas: `strip()` para evitar espacios invisibles.

Esto sigue el enfoque de `LimpiezaTransformacion.ipynb`.


In [ ]:

def parse_bp_series(series: pd.Series):
    # Convierte strings tipo '112/83' en dos series numéricas: sistólica y diastólica.
    sys_vals, dia_vals = [], []
    for v in series.astype(str):
        m = re.match(r"^\s*(\d+)\s*/\s*(\d+)\s*$", v)
        if m:
            sys_vals.append(float(m.group(1)))
            dia_vals.append(float(m.group(2)))
        else:
            sys_vals.append(np.nan)
            dia_vals.append(np.nan)
    return pd.Series(sys_vals, index=series.index), pd.Series(dia_vals, index=series.index)

def normalize_categories(df: pd.DataFrame, cols: list[str]) -> pd.DataFrame:
    df = df.copy()
    for c in cols:
        df[c] = df[c].astype("string").str.strip()
    return df

def clean_common(df: pd.DataFrame, cat_cols: list[str]) -> pd.DataFrame:
    df = df.copy()

    # Consistencia: quitar espacios en categóricas
    df = normalize_categories(df, cat_cols)

    # Validez: valores imposibles a NaN
    df.loc[df["Estimated LDL (mg/dL)"] < 0, "Estimated LDL (mg/dL)"] = np.nan
    df.loc[df["Total Cholesterol (mg/dL)"] < 0, "Total Cholesterol (mg/dL)"] = np.nan

    # Completar BP desde el string si es posible
    if "Blood Pressure (mmHg)" in df.columns:
        sys_bp, dia_bp = parse_bp_series(df["Blood Pressure (mmHg)"])
        df["Systolic BP"] = df["Systolic BP"].fillna(sys_bp)
        df["Diastolic BP"] = df["Diastolic BP"].fillna(dia_bp)

    return df

# Aplicar reglas de limpieza a train y test
train = train_raw.drop_duplicates().copy()
train = train.dropna(subset=[TARGET]).copy()
train = clean_common(train, cat_cols)

test = test_raw.drop_duplicates().copy()
test = clean_common(test, cat_cols)

print("Train limpio shape:", train.shape)
print("Test  limpio shape:", test.shape)

# Verificación: ahora LDL y colesterol negativos deberían ser 0
print("LDL negativo (train limpio):", (train["Estimated LDL (mg/dL)"] < 0).sum())
print("Colesterol total negativo (train limpio):", (train["Total Cholesterol (mg/dL)"] < 0).sum())


## 5. Exploración útil para modelado (visualizaciones)

Siguiendo `ManipulacionExploracionCalidad.ipynb`:
- Histograma del target
- Matriz de correlación numérica
- Top correlaciones con el target

Por qué: permite justificar ingeniería de características y entender qué variables parecen más relacionadas con el riesgo.


In [ ]:

plt.figure(figsize=(6,4))
sns.histplot(train[TARGET], kde=True)
plt.title("Distribución de CVD Risk Score")
plt.xlabel(TARGET)
plt.ylabel("Frecuencia")
plt.show()

corr = train.select_dtypes(include=[np.number]).corr()
plt.figure(figsize=(10,6))
sns.heatmap(corr, cmap="coolwarm", center=0)
plt.title("Heatmap de correlación (numéricas)")
plt.show()

target_corr = corr[TARGET].drop(TARGET).abs().sort_values(ascending=False)
display(target_corr.head(10))


## 6. Definir X, y y partición train-test (obligatorio del enunciado)

Qué haces:
- Separas X e y.
- Haces `train_test_split` con `random_state=42` y `test_size=0.25`.

Por qué: es una instrucción explícita del enunciado y crea el conjunto test para comparar modelos.


In [ ]:

DROP_ALWAYS = ["Patient ID", "Date of Service", AUX_LABEL]

X = train.drop(columns=[TARGET] + DROP_ALWAYS).copy()
y = train[TARGET].copy()

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE
)

print("X_train:", X_train.shape, "X_test:", X_test.shape)


## 7. Modelos: dos estrategias de preparación (con pipelines)

El enunciado pide al menos 2 modelos con preparación distinta.

Modelo 1 (baseline)
- Imputación + OHE
- Sin escalado numérico
- Sin ingeniería adicional

Modelo 2 (mejorado)
- Ingeniería de características simple (lineal)
  - Pulse Pressure = Systolic BP - Diastolic BP
  - Chol/HDL = Total Cholesterol / HDL
  - LDL/HDL = Estimated LDL / HDL
- Escalado numérico (StandardScaler)

Ambos modelos usan `Pipeline` como en `RegresionLineal.ipynb`.


In [ ]:

class FeatureEngineer(BaseEstimator, TransformerMixin):
    # Transformer scikit-learn que recibe DataFrame y devuelve DataFrame con columnas extra.
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()

        if "Systolic BP" in X.columns and "Diastolic BP" in X.columns:
            X["Pulse Pressure"] = X["Systolic BP"] - X["Diastolic BP"]

        if "Total Cholesterol (mg/dL)" in X.columns and "HDL (mg/dL)" in X.columns:
            X["Chol/HDL"] = X["Total Cholesterol (mg/dL)"] / X["HDL (mg/dL)"]

        if "Estimated LDL (mg/dL)" in X.columns and "HDL (mg/dL)" in X.columns:
            X["LDL/HDL"] = X["Estimated LDL (mg/dL)"] / X["HDL (mg/dL)"]

        return X

def build_preprocess(X_sample: pd.DataFrame, scale_numeric: bool) -> ColumnTransformer:
    num_cols = X_sample.select_dtypes(include=[np.number]).columns.tolist()
    cat_cols_local = [c for c in X_sample.columns if c not in num_cols]

    num_steps = [("imputer", SimpleImputer(strategy="median"))]
    if scale_numeric:
        num_steps.append(("scaler", StandardScaler()))

    preprocess = ColumnTransformer(
        transformers=[
            ("num", Pipeline(num_steps), num_cols),
            ("cat", Pipeline([
                ("imputer", SimpleImputer(strategy="most_frequent")),
                ("onehot", OneHotEncoder(handle_unknown="ignore", drop="first")),
            ]), cat_cols_local),
        ],
        remainder="drop"
    )
    return preprocess

# Modelo 1
preprocess_1 = build_preprocess(X_train, scale_numeric=False)
model_1 = Pipeline([
    ("preprocess", preprocess_1),
    ("reg", LinearRegression()),
])

# Modelo 2
X_train_fe = FeatureEngineer().transform(X_train)
preprocess_2 = build_preprocess(X_train_fe, scale_numeric=True)
model_2 = Pipeline([
    ("fe", FeatureEngineer()),
    ("preprocess", preprocess_2),
    ("reg", LinearRegression()),
])


## 8. Entrenar y evaluar los 2 modelos (RMSE, MAE, R2)

Qué haces:
- Entrenas en `X_train, y_train`.
- Predices en `X_test`.
- Calculas RMSE, MAE, R2 y construyes una tabla comparativa.

Por qué: corresponde a la evaluación cuantitativa del enunciado.


In [ ]:

def evaluate(model, X_tr, y_tr, X_te, y_te):
    model.fit(X_tr, y_tr)
    pred = model.predict(X_te)

    rmse = np.sqrt(mean_squared_error(y_te, pred))
    mae  = mean_absolute_error(y_te, pred)
    r2   = r2_score(y_te, pred)

    return rmse, mae, r2

rmse1, mae1, r21 = evaluate(model_1, X_train, y_train, X_test, y_test)
rmse2, mae2, r22 = evaluate(model_2, X_train, y_train, X_test, y_test)

results = pd.DataFrame([
    {"Modelo": "Modelo 1 (baseline)", "RMSE": rmse1, "MAE": mae1, "R2": r21},
    {"Modelo": "Modelo 2 (feat eng + scaler)", "RMSE": rmse2, "MAE": mae2, "R2": r22},
]).sort_values("RMSE", ascending=True)

display(results)

best_model_name = results.iloc[0]["Modelo"]
best_model = model_1 if best_model_name.startswith("Modelo 1") else model_2
print("Mejor modelo por RMSE:", best_model_name)


## 9. Interpretación del mejor modelo: coeficientes e importancia de variables

Qué haces:
- Extraes nombres de variables post preprocesamiento.
- Extraes coeficientes del regresor.
- Ordenas por valor absoluto.

Por qué: el enunciado exige coeficientes e importancia de variables.


In [ ]:

best_model.fit(X_train, y_train)

preprocess_step = best_model.named_steps["preprocess"]
feature_names = preprocess_step.get_feature_names_out()
coefs = best_model.named_steps["reg"].coef_

coef_df = (
    pd.DataFrame({"Feature": feature_names, "Coef": coefs})
      .assign(AbsCoef=lambda d: d["Coef"].abs())
      .sort_values("AbsCoef", ascending=False)
)

display(coef_df.head(25))


## 10. Verificación de supuestos (gráficos)

Qué haces:
- Residuos vs predicción: homocedasticidad y linealidad (visual).
- Histograma y QQ-plot: normalidad aproximada de errores.

Por qué: el enunciado pide validar supuestos para interpretación.


In [ ]:

y_pred = best_model.predict(X_test)
residuals = y_test - y_pred

plt.figure(figsize=(6,4))
plt.scatter(y_pred, residuals, alpha=0.4)
plt.axhline(0)
plt.title("Residuos vs Predicción")
plt.xlabel("Predicción")
plt.ylabel("Residuo (y - y_hat)")
plt.show()

plt.figure(figsize=(6,4))
sns.histplot(residuals, kde=True)
plt.title("Histograma de residuos")
plt.xlabel("Residuo")
plt.ylabel("Frecuencia")
plt.show()

import scipy.stats as stats
plt.figure(figsize=(6,4))
stats.probplot(residuals, dist="norm", plot=plt)
plt.title("QQ-plot de residuos")
plt.show()


### 10.1 Pruebas estadísticas opcionales (statsmodels)

Si quieres replicar el rigor de `RegresionLineal.ipynb`, instala statsmodels:
`pip install statsmodels`

Esta celda:
- Durbin-Watson: independencia aproximada
- Breusch-Pagan: homocedasticidad
- VIF: multicolinealidad


In [ ]:

try:
    import statsmodels.api as sm
    from statsmodels.stats.diagnostic import het_breuschpagan
    from statsmodels.stats.stattools import durbin_watson
    from statsmodels.stats.outliers_influence import variance_inflation_factor

    Xt_test = preprocess_step.transform(X_test)
    Xt_test_arr = Xt_test.toarray() if hasattr(Xt_test, "toarray") else Xt_test
    Xt_test_df = pd.DataFrame(Xt_test_arr, columns=feature_names, index=X_test.index)

    X_sm = sm.add_constant(Xt_test_df)
    model_sm = sm.OLS(y_test, X_sm).fit()

    dw = durbin_watson(model_sm.resid)
    print("Durbin-Watson:", dw)

    bp_test = het_breuschpagan(model_sm.resid, model_sm.model.exog)
    bp_labels = ["LM Stat", "LM p-value", "F Stat", "F p-value"]
    print("Breusch-Pagan:", dict(zip(bp_labels, bp_test)))

    vif_vals = []
    with np.errstate(divide="ignore", invalid="ignore"):
        for i in range(Xt_test_df.shape[1]):
            vif_vals.append(variance_inflation_factor(Xt_test_df.values, i))

    vif_df = pd.DataFrame({"Feature": feature_names, "VIF": vif_vals}).sort_values("VIF", ascending=False)
    display(vif_df.head(25))

except ImportError:
    print("No se pudo importar statsmodels. Si quieres estas pruebas: pip install statsmodels")


## 11. Entrenar el mejor modelo con TODO el train y predecir el test no etiquetado

Qué haces:
- Reentrenas con todo el train.
- Predices `CVD Risk Score` para el test.
- Exportas un CSV listo para entregar.

Por qué: es un entregable explícito del enunciado.


In [ ]:

X_full = train.drop(columns=[TARGET] + DROP_ALWAYS).copy()
y_full = train[TARGET].copy()

X_submit = test.drop(columns=DROP_ALWAYS).copy()

best_model.fit(X_full, y_full)
test_pred = best_model.predict(X_submit)

out = test_raw.copy()
out[TARGET] = test_pred

output_path = "Datos Test Lab 1.csv"
out.to_csv(output_path, index=False, sep=";")

print("Archivo generado para entrega:", output_path)
display(out.head())


## 12. Preguntas de análisis de resultados (responde aquí)

Responde directamente las preguntas del enunciado usando tus resultados:
1. Coeficientes del mejor modelo.
2. Mejor rendimiento y cómo interpretar RMSE, MAE y R2.
3. Variables seleccionadas e interpretación en el contexto.
4. Forma matemática de la regresión lineal y método (mínimos cuadrados ordinarios).
5. Dos tipos de sesgo que podrían afectar el resultado.
